## Load data

### Modules for the stock program

In [ ]:
!pip install scikit-optimize

     |████████████████████████████████| 102kB 5.5MB/s 


In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM,Dropout, Conv1D, Lambda, GRU
from tensorflow.keras.losses import Huber
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [ ]:
# window_size(전일 몇일치 데이터로 예측할 것인가)를 이용해서 LSTM에 필요한 데이터셋을 만들어주는 모듈
# Module for making batched data which can be used in LSTM Machine learning. - 4 columns
def making_batch(data_x, data_y, window_size):
  # 사용법 : X_data_batch, y_data_batch = making_batch(data_x, data_y, 50)
  # ndarray 만을 넣어야 함.
  new_data_x = []
  new_data_y = []
  for i in range(len(data_x) - window_size):
    _x = data_x[i: i+window_size]
    _y = data_y[i+window_size]
    new_data_x.append(_x)
    new_data_y.append(_y)
  new_data_x = np.array(new_data_x)
  new_data_y = np.array(new_data_y)
  return new_data_x, new_data_y

# 종가만으로 batch 만드는 프로그램. - 1 columns (only the close price)
def making_batch_pr(data_y, window_size):
  new_data_x = []
  new_data_y = []
  for i in range(len(data) - window_size):
    _x = data_y[i: i+window_size] # 종가 20일치
    _y = data_y[i+window_size] # 그다음 하루의 종가
    new_data_x.append(_x) # 20일치 x에 넣고
    new_data_y.append(_y) # 1일치 y에 넣는다.
  return new_data_x, new_data_y
# window_size 만큼의 x 데이터의 결과값이 y[window_size+1] 이 된다.
# 즉 그 전날 60일만큼의 데이터가 그 다음의 주가를 예측한다.

In [ ]:
# 데이터를 가져오는 함수# 데이터를 가져오는 모듈
def getting_data(need_date='all'):
  # need_date 로는 필요한 일수를 입력한다.
  samsung = pd.read_csv('./A005930.csv')
  cols = ['DAY','CUR_PR','HIGH_PR','LOW_PR','CLO_PR','FOR_STOR']
  samsung = samsung[cols]
  samsung = samsung.rename(columns = {'DAY':'date','CUR_PR':'open','HIGH_PR':'high','LOW_PR':'low','CLO_PR':'close','FOR_STOR':'volume'})
  samsung.sort_values(by='date', inplace=True)
  samsung.reset_index(drop=True, inplace=True)
  if need_date=='all':
    return samsung
  else:
    samsung = samsung[-need_date:]
    samsung.reset_index(drop=True, inplace=True)
    return samsung
  

In [ ]:
# 기존 모델
def create_model_GRU(input_shape, dropout_rate,
                 num_first_LSTM_nodes, num_second_LSTM_nodes, 
                 activation='relu',learning_rate=0.00001):
  model = Sequential()
  model.add(GRU(units = num_first_LSTM_nodes, activation=activation, return_sequences = True, input_shape=input_shape))
  # LSTM 1층
  #model.add(Dropout(dropout_rate))
  # 드롭아웃 층
  model.add(GRU(units = num_second_LSTM_nodes, activation=activation))
  # LSTM 2층
  #model.add(Dropout(dropout_rate))
  model.add(Dense(units = 1)) # 출력증

  adam = Adam(lr=learning_rate)
  loss=Huber()
  model.compile(optimizer=adam, loss=loss)
  return model

In [ ]:
# 모델 구조 만드는 모듈
# the usual imports for a vanilla nueral net
import keras
from keras.models import Sequential
from keras.layers import Dense
import tensorflow
from tensorflow.python.keras import backend as K
from keras.optimizers import Adam

In [ ]:
window_size = 30

In [ ]:
def create_model(learning_rate, num_dense_layers,num_input_nodes,
                 num_dense_nodes, activation, adam_decay): # adam_decay - 학습율 조정과 관련된 파라미터.
    #start the model making process and create our first layer
    model = Sequential()
    # input shape 에 (window_size, column 갯수) 를 넣어준다.
    model.add(LSTM(units = num_input_nodes, input_shape= (30,4), activation=activation, return_sequences=True))
    # input_shape 을 따로 넣어줘야 한다.
    # 첫번째 층 완료.

    #create a loop making a new dense layer for the amount passed to this model.
    #naming the layers helps avoid tensorflow error deep in the stack trace.
    for i in range(num_dense_layers):
      # 몇층으로 할지,
        name = 'layer_dense_{0}'.format(i+1)
        model.add(Dense(num_dense_nodes,
                 activation=activation,
                        name=name
                 ))
    #add our classification layer.
    model.add(Dense(units=1))
    # 여기 노드수 1로 해야 하고, 활성화함수는 없어야 함.
    
    #setup our optimizer and compile
    adam = Adam(lr=learning_rate, decay= adam_decay)
    loss=Huber()
    model.compile(optimizer=adam, loss=loss,
                 metrics=['accuracy'])
    # 주식은 loss 에 Huber() 객체를 넣어주는 듯 함.
    return model


In [ ]:
#imports we know we'll need
import skopt
from skopt import gbrt_minimize, gp_minimize
from skopt.utils import use_named_args
from skopt.space import Real, Categorical, Integer

In [ ]:
# skopt 안의 space 는 범위를 지정해주는 틀이다.
# 지정해주면 그 사이 값으로 skopt 모듈의 최적화기가 무작위로 알아서 넣어준다.
dim_learning_rate = Real(low=1e-4, high=1e-1, prior='log-uniform',
                         name='learning_rate')
dim_num_dense_layers = Integer(low=1, high=5, name='num_dense_layers')
dim_num_input_nodes = Integer(low=1, high=512, name='num_input_nodes')
dim_num_dense_nodes = Integer(low=1, high=28, name='num_dense_nodes')
dim_activation = Categorical(categories=['relu', 'sigmoid'],
                             name='activation')
dim_batch_size = Integer(low=1, high=128, name='batch_size')
dim_adam_decay = Real(low=1e-6,high=1e-2,name="adam_decay")

dimensions = [dim_learning_rate,
              dim_num_dense_layers,
              dim_num_input_nodes,
              dim_num_dense_nodes,
              dim_activation,
              dim_batch_size,
              dim_adam_decay
             ]
default_parameters = [1e-3, 1,512, 13, 'relu',64, 1e-3]

In [ ]:
samsung = getting_data(700)
samsung

cols_for_scaling = ['open','high','low','close','volume']
scaler = MinMaxScaler()
scaled_samsung = scaler.fit_transform(samsung[cols_for_scaling])
scaled_samsung = pd.DataFrame(scaled_samsung, columns=cols_for_scaling, index=samsung['date'])
scaled_samsung

data_cols = ['open','high','low','volume']
target_cols = ['close']
X_data = scaled_samsung[data_cols]
y_data = scaled_samsung[target_cols]
window_size=30

new_X_data, new_y_data = making_batch(X_data.values, y_data.values, window_size)
print("X:",new_X_data.shape)
print("y:",new_y_data.shape)

input_shape = new_X_data.shape[1:]

# 최근 100일이 테스트셋, 100일 전까지의 데이터는 트레이닝셋
test_date = 100
X_train = new_X_data[:-test_date]
X_test = new_X_data[-test_date:]
y_train = new_y_data[:-test_date]
y_test = new_y_data[-test_date:]

# # train_data_set 과 val_data_set 을 나누자.
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, shuffle=False)

X: (670, 30, 4)
y: (670, 1)


In [ ]:
@use_named_args(dimensions=dimensions)
def fitness(learning_rate, num_dense_layers, num_input_nodes, 
            num_dense_nodes,activation, batch_size,adam_decay):
  # fitting(훈련) 시키는 메소드인 듯.

    model = create_model(
                         learning_rate=learning_rate, # 최적화기의 학습율 지정
                         num_dense_layers=num_dense_layers, # 밀집층을 몇개의 층으로 할 거냐.
                         num_input_nodes=num_input_nodes, # 입력층의 노드 갯수 지정
                         num_dense_nodes=num_dense_nodes, # 밀집층(히든층)의 노드 갯수 지정
                         activation=activation, # 밀집층의 활성화함수로는 뭘 쓸 것이냐 (주식에선 보통 relu 씀)
                         adam_decay=adam_decay # 이건 뭐지??
                        )
    

    #named blackbox becuase it represents the structure
    # 만든 모델로 학습을 진행해준다.
    blackbox = model.fit(x=X_train, # 훈련 데이터를 넣어준다.
                        y=y_train, # 정답을 넣어준다.
                        epochs=3, # 몇회 돌릴 것인지 넣어준다.
                        batch_size=batch_size, # 배치 사이즈는 몇개로 할 것인지 (부분집합 갯수)
                        validation_split=0.15, # validation 데이터로 얼마나 분할할 것인지.
                        )
    #return the validation accuracy for the last epoch.
    loss = blackbox.history['val_loss'][-1] # val_accuracy 를 따로 담아준다. (출력하여 보기 위함)

    # Print the classification accuracy.
    print()
    print("loss: {0:.2}".format(loss))
    print()


    # Delete the Keras model with these hyper-parameters from memory.
    del model 
    
    # Clear the Keras session, otherwise it will keep adding new
    # models to the same TensorFlow graph each time we create
    # a model with a different set of hyper-parameters.
    K.clear_session()
    tensorflow.compat.v1.reset_default_graph()
    # 이거 안하면 하이퍼파라미터 초기화가 안되서
    # 새로운 텐서플로우 모델이 생기지 않음.
    # 그래서 항상 clear 해줘야 함.
    
    return loss

In [ ]:
# 이 코드를 텐서플로우에 항상 넣어주는게 좋다는데?
# 그래야 문제가 안일어난데.
K.clear_session()
tensorflow.compat.v1.reset_default_graph()

In [ ]:
gp_result = gp_minimize(func=fitness,
                            dimensions=dimensions,
                            n_calls=12,
                            noise= 0.01,
                            n_jobs=-1,
                            kappa = 5,
                            x0=default_parameters)
# dimensions 안에 파라미터들이 다 들어가 있음.
# fitness 의 출력물로는 -1*정확도가 나옴.
# 즉, -1*정확도가 최소화가 되는 방향이 곧 정확도가 가장 높은 방향이다.
# 그래서 gp_minimize 라는 함수를 통해 정확도의 최대값을 구한다.

Epoch 1/3
8/8 [==============================] - 3s 146ms/step - loss: 0.0087 - accuracy: 0.0032 - val_loss: 0.0144 - val_accuracy: 0.0000e+00
Epoch 2/3
8/8 [==============================] - 1s 112ms/step - loss: 0.0031 - accuracy: 0.0018 - val_loss: 0.0060 - val_accuracy: 0.0000e+00
Epoch 3/3
8/8 [==============================] - 1s 105ms/step - loss: 0.0027 - accuracy: 0.0018 - val_loss: 0.0044 - val_accuracy: 0.0000e+00

loss: 0.0044

Epoch 1/3
6/6 [==============================] - 2s 155ms/step - loss: 83.0586 - accuracy: 9.0429e-04 - val_loss: 0.0872 - val_accuracy: 0.0000e+00
Epoch 2/3
6/6 [==============================] - 0s 84ms/step - loss: 0.0507 - accuracy: 0.0026 - val_loss: 0.0338 - val_accuracy: 0.0000e+00
Epoch 3/3
6/6 [==============================] - 1s 88ms/step - loss: 0.0078 - accuracy: 0.0018 - val_loss: 0.0152 - val_accuracy: 0.0000e+00

loss: 0.015

Epoch 1/3
6/6 [==============================] - 3s 188ms/step - loss: 5.0128 - accuracy: 0.0042 - val_loss: 0

In [ ]:
gp_result.fun

0.001606404664926231

In [ ]:
gp_result

          fun: 0.001606404664926231
    func_vals: array([0.00439719, 0.01519386, 0.0075621 , 0.00390762, 0.03622856,
       0.1901671 , 0.01554267, 0.00233966, 0.00304531, 0.0016064 ,
       0.03458865, 0.05739554])
       models: [GaussianProcessRegressor(alpha=1e-10, copy_X_train=True,
                         kernel=1**2 * Matern(length_scale=[1, 1, 1, 1, 1, 1, 1], nu=2.5) + WhiteKernel(noise_level=0.01),
                         n_restarts_optimizer=2, noise=0.01, normalize_y=True,
                         optimizer='fmin_l_bfgs_b', random_state=767162376), GaussianProcessRegressor(alpha=1e-10, copy_X_train=True,
                         kernel=1**2 * Matern(length_scale=[1, 1, 1, 1, 1, 1, 1], nu=2.5) + WhiteKernel(noise_level=0.01),
                         n_restarts_optimizer=2, noise=0.01, normalize_y=True,
                         optimizer='fmin_l_bfgs_b', random_state=767162376)]
 random_state: RandomState(MT19937) at 0x7F9DFC6E3050
        space: Space([Real(low=0.0001, hi

In [ ]:
print("best loss was " + str((gp_result.fun)))

best loss was 0.001606404664926231


In [ ]:
gp_result.x # 최적의 하이퍼파라미터를 출력해준다.

[0.022207201170733794, 3, 277, 3, 'sigmoid', 118, 0.004802287735881647]

In [ ]:
gp_result.func_vals # 최적의 하이퍼 파라미터로 했을 때 교차검증(또는 에포크수에 따른)한 정확도들을 보여준다.

array([0.00439719, 0.01519386, 0.0075621 , 0.00390762, 0.03622856,
       0.1901671 , 0.01554267, 0.00233966, 0.00304531, 0.0016064 ,
       0.03458865, 0.05739554])

In [ ]:
import pandas as pd
pd.concat([pd.DataFrame(gp_result.x_iters, columns = ["learning rate","hidden layers","input layer nodes","hidden layer nodes",
                                           "activation function","batch size","adam learning rate decay"]),
(pd.Series(gp_result.func_vals, name="loss"))], axis=1)

,learning rate,hidden layers,input layer nodes,hidden layer nodes,activation function,batch size,adam learning rate decay,loss
0,0.001000,1,512,13,relu,64,0.001000,0.004397
1,0.026973,1,146,5,relu,91,0.001693,0.015194
2,0.008573,4,321,13,relu,90,0.003081,0.007562
3,0.010323,2,29,16,sigmoid,116,0.002378,0.003908
4,0.000918,4,476,9,relu,49,0.009243,0.036229
5,0.002954,4,494,8,sigmoid,31,0.004815,0.190167
6,0.000278,5,502,15,sigmoid,8,0.002212,0.015543
7,0.040679,5,87,20,sigmoid,102,0.004237,0.002340
8,0.007216,1,448,12,relu,34,0.008763,0.003045
9,0.022207,3,277,3,sigmoid,118,0.004802,0.001606


[0.001, 1, 512, 13, 'relu', 64, 0.001]

Creating our search parameters.
"dim_" short for dimension. Its' just a way to label our parameters.

We can search across nearly every param in a keras model. 
This code focuses on: 
* Number of Layers
* Number of Nodes per layer
* Learning Rate & Weight Decay for the Adam Optimizer
* activation functions
* batch size

The name feature allows us to use the `@use_named_args` decorator.
We must also establish default parameters. 

In [ ]:
# 이 코드를 텐서플로우에 항상 넣어주는게 좋다는데?
# 그래야 문제가 안일어난데.
K.clear_session()
tensorflow.compat.v1.reset_default_graph()

In [ ]:
# The explanation for the parameters in gp_minimize method

gp_result = gp_minimize(func=fitness,
                            dimensions=dimensions,
                            n_calls=12,
                            noise= 0.01,
                            n_jobs=-1,
                            kappa = 5,
                            x0=default_parameters)
# dimensions 안에 파라미터들이 다 들어가 있음.
# fitness 의 출력물로는 -1*정확도가 나옴.
# 즉, -1*정확도가 최소화가 되는 방향이 곧 정확도가 가장 높은 방향이다.
# 그래서 gp_minimize 라는 함수를 통해 정확도의 최대값을 구한다.

Epoch 1/3
797/797 [==============================] - 4s 4ms/step - loss: 0.5460 - accuracy: 0.8372 - val_loss: 0.1344 - val_accuracy: 0.9634
Epoch 2/3
797/797 [==============================] - 3s 4ms/step - loss: 0.1314 - accuracy: 0.9613 - val_loss: 0.1101 - val_accuracy: 0.9670
Epoch 3/3
797/797 [==============================] - 3s 4ms/step - loss: 0.0855 - accuracy: 0.9765 - val_loss: 0.0885 - val_accuracy: 0.9751

Accuracy: 97.51%

Epoch 1/3
1822/1822 [==============================] - 8s 4ms/step - loss: 1.1361 - accuracy: 0.6138 - val_loss: 0.3122 - val_accuracy: 0.9106
Epoch 2/3
1822/1822 [==============================] - 7s 4ms/step - loss: 0.3289 - accuracy: 0.9052 - val_loss: 0.2524 - val_accuracy: 0.9294
Epoch 3/3
1822/1822 [==============================] - 7s 4ms/step - loss: 0.2730 - accuracy: 0.9212 - val_loss: 0.2302 - val_accuracy: 0.9358

Accuracy: 93.58%

Epoch 1/3
981/981 [==============================] - 5s 4ms/step - loss: 0.9781 - accuracy: 0.6593 - val_loss:

## Find our best accuracy

In [ ]:
print("best accuracy was " + str(round(gp_result.fun *-100,2))+"%.")

best accuracy was 97.51%.


### returning the parameters for the best function

In [ ]:
gp_result.x # 최적의 하이퍼파라미터를 출력해준다.

[0.001, 1, 512, 13, 'relu', 64, 0.001]

In [ ]:
gp_result.func_vals # 최적의 하이퍼 파라미터로 했을 때 교차검증(또는 에포크수에 따른)한 정확도들을 보여준다.

array([-0.97511113, -0.93577778, -0.93077779, -0.96944445, -0.5777778 ,
       -0.10633333, -0.94266665, -0.97088891, -0.76566666, -0.44311112,
       -0.93944442, -0.24622223])

## Trying a gradient boosted search with a simpler model

In [ ]:
# Using gbrt (which is a optimization technology with boosting technology)
gbrt_result = gbrt_minimize(func=fitness,
                            dimensions=dimensions,
                            n_calls=12,
                            n_jobs=-1,
                            x0=default_parameters)


In [ ]:
print("best accuracy was " + str(round(gbrt_result.fun *100,2))+"%.")

In [ ]:
pd.concat([pd.DataFrame(gbrt_result.x_iters, columns = ["learning rate","hidden layers","input layer nodes","hidden layer nodes",
                                           "activation function","batch size","adam learning rate decay"]),
(pd.Series(gbrt_result.func_vals*-100, name="accuracy"))], axis=1)

In [ ]:
gbrt_result.x

In [ ]:
K.clear_session()
tensorflow.reset_default_graph()

In [ ]:
#call our best model 
gbrt_model = create_model(gbrt_result.x[0],gbrt_result.x[1],gbrt_result.x[2],gbrt_result.x[3],gbrt_result.x[4],gbrt_result.x[5])
gbrt_model.summary()
#retrain our best model architecture
model.fit(X_train,y_train, epochs=3)
model.evaluate(X_test,y_test)